In [4]:
import pandas as pd
import numpy as np


#users_list = [i.strip().split("::") for i in open('raw/users.dat', 'r').readlines()]
#movies_list = [i.strip().split("::") for i in open('raw/u.genre', 'r').readlines()]
ratings_df = pd.read_csv('raw/u.data', sep='\t', header=None, names=['UserID', 'MovieID', 'Rating', 'Timestamp'])
movies_df = pd.read_csv('raw/u.genres', columns = ['MovieID', 'Title', 'Genres'])
#movies_df['MovieID'] = movies_df['MovieID'].apply(pd.to_numeric)

In [5]:
R_df = ratings_df.pivot(index = 'UserID', columns ='MovieID', values = 'Rating').fillna(0)
R_df.head()

MovieID,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
UserID,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
R = R_df.as_matrix()
user_ratings_mean = np.mean(R, axis = 1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)

C:\Users\abhin\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


## Predicting using singular value decomposition

In [7]:
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(R_demeaned, k = 50)

In [8]:
sigma = np.diag(sigma)

In [9]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = R_df.columns)

In [12]:
preds_df.head()

MovieID,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
0,6.488436,2.959503,1.634987,3.024467,1.656526,1.659506,3.630469,0.240669,1.791518,3.347816,...,0.011976,-0.092017,-0.074553,-0.060985,0.009427,-0.035641,-0.039227,-0.037434,-0.025552,0.023513
1,2.347262,0.129689,-0.098917,0.328828,0.159517,0.481361,0.213002,0.097908,1.892100,0.671000,...,0.003943,-0.026939,-0.035460,-0.029883,-0.027153,-0.015244,-0.008277,-0.011760,0.011639,-0.046924
2,0.291905,-0.263830,-0.151454,-0.179289,0.013462,-0.088309,-0.057624,0.568764,-0.018506,0.280742,...,-0.028964,-0.031622,0.045513,0.026089,-0.021705,0.002282,0.032363,0.017322,-0.006644,-0.009480
3,0.366410,-0.443535,0.041151,-0.007616,0.055373,-0.080352,0.299015,-0.010882,-0.160888,-0.118834,...,0.020069,0.015981,-0.000182,0.005593,0.026634,0.023562,0.036405,0.029984,0.015612,-0.008713
4,4.263488,1.937122,0.052529,1.049350,0.652765,0.002836,1.730461,0.870584,0.341027,0.569055,...,0.019973,-0.053521,-0.017242,-0.007137,-0.038987,0.010338,0.004869,0.007603,-0.020575,0.003330


In [11]:
def recommend_movies(predictions_df, userID, movies_df, original_ratings_df, num_recommendations=5):
    
    # Get and sort the user's predictions
    user_row_number = userID - 1 # UserID starts at 1, not 0
    sorted_user_predictions = predictions_df.iloc[user_row_number].sort_values(ascending=False)
    
    # Get the user's data and merge in the movie information.
    user_data = original_ratings_df[original_ratings_df.UserID == (userID)]
    user_full = (user_data.merge(movies_df, how = 'left', left_on = 'MovieID', right_on = 'MovieID').
                     sort_values(['Rating'], ascending=False)
                 )

    print('User {0} has already rated {1} movies.'.format(userID, user_full.shape[0]))
    print('Recommending the highest {0} predicted ratings movies not already rated.'.format(num_recommendations))
    
    # Recommend the highest predicted rating movies that the user hasn't seen yet.
    recommendations = (movies_df[~movies_df['MovieID'].isin(user_full['MovieID'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'MovieID',
               right_on = 'MovieID').
         rename(columns = {user_row_number: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )

    return user_full, recommendations

In [16]:
movie_df = pd.read_csv('raw/u.item', sep='|', header=None, encoding='latin1', index_col=False,
                   names=['MovieID',
                          'movie_title',
                          'release_date',
                          'video_release_date',
                          'imdb_url',
                          'unknown',
                          'action',
                          'adventure',
                          'animation',
                          'children',
                          'comedy',
                          'crime',
                          'documentary',
                          'drama',
                          'fantasy',
                          'film_noir',
                          'horror',
                          'musical',
                          'mystery'
                          'romance',
                          'sci_fi',
                          'thriller',
                          'war',
                          'western'])
movie_df.head()

,MovieID,movie_title,release_date,video_release_date,imdb_url,unknown,action,adventure,animation,children,...,drama,fantasy,film_noir,horror,musical,mysteryromance,sci_fi,thriller,war,western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,0,1,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0


In [21]:
already_rated, predictions = recommend_movies(preds_df, 200, movie_df, ratings_df, 10)

User 200 has already rated 216 movies.
Recommending the highest 10 predicted ratings movies not already rated.


In [22]:
already_rated.head(10)

,UserID,MovieID,Rating,Timestamp,movie_title,release_date,video_release_date,imdb_url,unknown,action,...,drama,fantasy,film_noir,horror,musical,mysteryromance,sci_fi,thriller,war,western
0,200,222,5,876042340,Star Trek: First Contact (1996),22-Nov-1996,NaN,http://us.imdb.com/M/title-exact?Star%20Trek:%...,0,1,...,0,0,0,0,0,0,0,1,0,0
165,200,385,5,884129696,True Lies (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?True%20Lies%2...,0,1,...,0,0,0,0,0,0,1,0,0,0
74,200,228,5,884128372,Star Trek: The Wrath of Khan (1982),01-Jan-1982,NaN,http://us.imdb.com/M/title-exact?Star%20Trek:%...,0,1,...,0,0,0,0,0,0,0,1,0,0
148,200,568,5,884128372,Speed (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?Speed%20(1994/I),0,1,...,0,0,0,0,0,0,1,0,1,0
154,200,423,5,884129275,E.T. the Extra-Terrestrial (1982),01-Jan-1982,NaN,http://us.imdb.com/M/title-exact?E%2ET%2E%20th...,0,0,...,1,1,0,0,0,0,0,1,0,0
65,200,183,5,884128554,Alien (1979),01-Jan-1979,NaN,http://us.imdb.com/M/title-exact?Alien%20(1979),0,1,...,0,0,0,1,0,0,0,1,1,0
156,200,117,5,876042268,"Rock, The (1996)",07-Jun-1996,NaN,"http://us.imdb.com/M/title-exact?Rock,%20The%2...",0,1,...,0,0,0,0,0,0,0,0,1,0
158,200,496,5,884128904,It's a Wonderful Life (1946),01-Jan-1946,NaN,http://us.imdb.com/M/title-exact?It's%20a%20Wo...,0,0,...,1,0,0,0,0,0,0,0,0,0
162,200,483,5,884128426,Casablanca (1942),01-Jan-1942,NaN,http://us.imdb.com/M/title-exact?Casablanca%20...,0,0,...,1,0,0,0,0,0,1,0,0,1
164,200,132,5,884130792,"Wizard of Oz, The (1939)",01-Jan-1939,NaN,http://us.imdb.com/M/title-exact?Wizard%20of%2...,0,0,...,1,0,0,0,1,0,0,0,0,0


In [23]:
predictions.head(10)

,MovieID,movie_title,release_date,video_release_date,imdb_url,unknown,action,adventure,animation,children,...,drama,fantasy,film_noir,horror,musical,mysteryromance,sci_fi,thriller,war,western
286,403,Batman (1989),01-Jan-1989,NaN,http://us.imdb.com/M/title-exact?Batman%20(1989),0,1,1,0,0,...,1,0,0,0,0,0,0,0,0,0
413,566,Clear and Present Danger (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?Clear%20and%2...,0,1,1,0,0,...,0,0,0,0,0,0,0,0,1,0
86,133,Gone with the Wind (1939),01-Jan-1939,NaN,http://us.imdb.com/M/title-exact?Gone%20with%2...,0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,1
381,526,Ben-Hur (1959),01-Jan-1959,NaN,http://us.imdb.com/M/title-exact?Ben-Hur%20(1959),0,1,1,0,0,...,1,0,0,0,0,0,0,0,0,0
7,14,"Postino, Il (1994)",01-Jan-1994,NaN,"http://us.imdb.com/M/title-exact?Postino,%20Il...",0,0,0,0,0,...,1,0,0,0,0,0,1,0,0,0
117,180,Apocalypse Now (1979),01-Jan-1979,NaN,http://us.imdb.com/M/title-exact?Apocalypse%20...,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
377,521,"Deer Hunter, The (1978)",01-Jan-1978,NaN,http://us.imdb.com/M/title-exact?Deer%20Hunter...,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
121,185,Psycho (1960),01-Jan-1960,NaN,http://us.imdb.com/M/title-exact?Psycho%20(1960),0,0,0,0,0,...,0,0,0,1,0,0,1,0,1,0
400,550,Die Hard: With a Vengeance (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Die%20Hard:%2...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
403,554,Waterworld (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Waterworld%20...,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


## Average number of likes